In [9]:
import numpy as np
import plotly.graph_objects as go
from policyengine_us import Simulation
from policyengine_core.reforms import Reform
from policyengine_core.charts import format_fig

In [10]:
YEAR = "2024"
MAX_INCOME = 150000

In [11]:
def create_situation(scenario):
    base_situation = {
        "axes": [[
            {
                "name": "employment_income",
                "min": 0,
                "max": MAX_INCOME,
                "count": 201,
                "period": YEAR,
            }
        ]]
    }
    
    if scenario == "TX":
        situation = {
            "families": {"your family": {"members": ["you", "your first dependent", "your second dependent"]}},
            "households": {
                "your household": {
                    "members": ["you", "your first dependent", "your second dependent"],
                    "safmr_used_for_hcv": {YEAR: 0},
                    "small_area_fair_market_rent": {YEAR: 19080},
                    "state_name": {YEAR: "TX"}
                }
            },
            "marital_units": {
                "your marital unit": {"members": ["you"]},
                "your first dependent's marital unit": {"members": ["your first dependent"], "marital_unit_id": {YEAR: 1}},
                "your second dependent's marital unit": {"members": ["your second dependent"], "marital_unit_id": {YEAR: 2}}
            },
            "people": {
                "you": {"age": {YEAR: 40}, "pre_subsidy_rent": {YEAR: 18000}},
                "your first dependent": {"age": {YEAR: 10}},
                "your second dependent": {"age": {YEAR: 10}}
            },
            "spm_units": {"your household": {"members": ["you", "your first dependent", "your second dependent"]}},
            "tax_units": {"your tax unit": {"members": ["you", "your first dependent", "your second dependent"]}}
        }
    elif scenario == "MO":
        situation = {
            "families": {"your family": {"members": ["you"]}},
            "households": {
                "your household": {
                    "members": ["you"],
                    "safmr_used_for_hcv": {YEAR: 0},
                    "small_area_fair_market_rent": {YEAR: 6480},
                    "state_name": {YEAR: "MO"}
                }
            },
            "marital_units": {"your marital unit": {"members": ["you"]}},
            "people": {"you": {"age": {YEAR: 40}, "pre_subsidy_rent": {YEAR: 6480}}},
            "spm_units": {"your household": {"members": ["you"]}},
            "tax_units": {"your tax unit": {"members": ["you"]}}
        }
    elif scenario == "CA":
        situation = {
            "families": {"your family": {"members": ["you", "your first dependent", "your second dependent"]}},
            "households": {
                "your household": {
                    "members": ["you", "your first dependent", "your second dependent"],
                    "safmr_used_for_hcv": {YEAR: 1},
                    "small_area_fair_market_rent": {YEAR: 83520},
                    "state_name": {YEAR: "CA"}
                }
            },
            "marital_units": {
                "your marital unit": {"members": ["you"]},
                "your first dependent's marital unit": {"members": ["your first dependent"], "marital_unit_id": {YEAR: 1}},
                "your second dependent's marital unit": {"members": ["your second dependent"], "marital_unit_id": {YEAR: 2}}
            },
            "people": {
                "you": {"age": {YEAR: 40}, "pre_subsidy_rent": {YEAR: 83520}},
                "your first dependent": {"age": {YEAR: 10}},
                "your second dependent": {"age": {YEAR: 3}}
            },
            "spm_units": {"your household": {"members": ["you", "your first dependent", "your second dependent"]}},
            "tax_units": {"your tax unit": {"members": ["you", "your first dependent", "your second dependent"]}}
        }
    
    situation.update(base_situation)
    return situation

In [12]:
def calculate_rent_relief_credit(scenario, reform=None):
    situation = create_situation(scenario)
    
    if reform:
        simulation = Simulation(reform=reform, situation=situation)
    else:
        simulation = Simulation(situation=situation)
    
    return simulation.calculate("rent_relief_tax_credit", YEAR)


In [23]:
def create_rent_relief_comparison_graph():
    reform = Reform.from_dict({
        "gov.contrib.harris.rent_relief_act.rent_relief_credit.in_effect": {
            f"{YEAR}-01-01.2100-12-31": True
        }
    }, country_id="us")

    x = np.linspace(0, MAX_INCOME, 201)
    scenarios = [
        ("TX", "Texas 3bd"),
        ("MO", "Missouri studio"),
        ("CA", "San Diego 4bd")
    ]
    colors = {
        "TX": "#0066cc",  # Light blue
        "MO": "#4d94ff",  # Medium blue
        "CA": "#003366",  # Dark blue
    }

    fig = go.Figure()

    for scenario_code, scenario_label in scenarios:
        credit = calculate_rent_relief_credit(scenario_code, reform)

        fig.add_trace(go.Scatter(
            x=x,
            y=credit,
            mode='lines',
            name=scenario_label,
            line=dict(color=colors[scenario_code], width=2)
        ))

    fig.update_layout(
        title='Rent Relief Tax Credit Impact',
        xaxis_title="Earnings",
        yaxis_title="Credit Amount",
        xaxis=dict(tickformat='$,.0f', range=[0, MAX_INCOME]),
        yaxis=dict(tickformat='$,.0f'),
        legend=dict(
            yanchor="top",
            y=0.99,
            xanchor="left",
            x=1.01
        ),
        height=600,
        width=800,
        plot_bgcolor='white',
        paper_bgcolor='white',
    )

    fig.update_xaxes(gridcolor='lightgray', zeroline=True, zerolinecolor='lightgray')
    fig.update_yaxes(gridcolor='lightgray', zeroline=True, zerolinecolor='lightgray')

    return fig

In [24]:
# Create and display the chart
fig = create_rent_relief_comparison_graph()
fig = format_fig(fig)
fig.show()